<a href="https://colab.research.google.com/github/ipeirotis/dealing_with_data/blob/master/01-Pandas/B2a-Inserting_Data_in_MySQL_using_PythonPandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Inserting and Reading data from MySQL using Pandas

First let's start with a basic piece of code that fetches the data that we want to insert in the database. For our example, we will get the data about the Citibike stations, using the correspoding API call provided by the Citibike website:

In [1]:
!pip install -U -q PyMySQL sqlalchemy sql_magic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 787.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 7.7 MB/s eta 0:00:00


In [2]:
import requests

In [3]:
# Let's get the data from the Citibike API

# This gives information for each station that remains stable over time
url_stations = "https://gbfs.citibikenyc.com/gbfs/en/station_information.json"
# This gives the live status of all the stations (e.g., bikes available etc)
url_status = "https://gbfs.citibikenyc.com/gbfs/en/station_status.json"

# We fetch for now just the time-invariant data
results = requests.get(url_stations).json()

In [4]:
# We only need a subset of the data in the JSON returned by the Citibike API, so we keep only what we need
data = results["data"]["stations"]

In [5]:
len(data)

2098

In [6]:
import pandas as pd

df = pd.DataFrame(data)
df.head(5)

,capacity,rental_uris,has_kiosk,lat,rental_methods,eightd_station_services,lon,station_id,external_id,electric_bike_surcharge_waiver,region_id,name,station_type,eightd_has_key_dispenser,short_name
0,87,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",True,40.750756,"[KEY, CREDITCARD]",[],-73.978326,c638ec67-9ac0-416f-944f-619926144931,c638ec67-9ac0-416f-944f-619926144931,False,71,E 40 St & Park Ave,classic,False,6432.11
1,30,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",True,40.763605,"[KEY, CREDITCARD]",[],-73.989180,b442a648-e9f4-4893-951a-64d258bc0e55,b442a648-e9f4-4893-951a-64d258bc0e55,False,71,W 50 St & 9 Ave,classic,False,6854.05
2,0,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",True,40.737150,"[KEY, CREDITCARD]",[],-73.856140,1851254468343716806,1851254468343716806,False,NaN,60 Ave & Otis Ave,classic,False,5945.01
3,0,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",True,40.736490,"[KEY, CREDITCARD]",[],-73.869950,1846086362824884662,1846086362824884662,False,NaN,56 Ave & 92 St,classic,False,5962.04
4,103,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",True,40.760177,"[KEY, CREDITCARD]",[],-73.984868,64f0f28c-bedc-42d5-b107-ecdd48fc30cd,64f0f28c-bedc-42d5-b107-ecdd48fc30cd,False,71,Broadway & W 48 St,classic,False,6708.04


In [7]:
# We drop the 'rental methods' columns,
# as they contains multiple values and
# we cannot insert lists in a database cell.
df.drop(
    ["rental_methods", "eightd_station_services", "rental_uris"],
    axis="columns",
    inplace=True,
)
df.head(5)

,capacity,has_kiosk,lat,lon,station_id,external_id,electric_bike_surcharge_waiver,region_id,name,station_type,eightd_has_key_dispenser,short_name
0,87,True,40.750756,-73.978326,c638ec67-9ac0-416f-944f-619926144931,c638ec67-9ac0-416f-944f-619926144931,False,71,E 40 St & Park Ave,classic,False,6432.11
1,30,True,40.763605,-73.989180,b442a648-e9f4-4893-951a-64d258bc0e55,b442a648-e9f4-4893-951a-64d258bc0e55,False,71,W 50 St & 9 Ave,classic,False,6854.05
2,0,True,40.737150,-73.856140,1851254468343716806,1851254468343716806,False,NaN,60 Ave & Otis Ave,classic,False,5945.01
3,0,True,40.736490,-73.869950,1846086362824884662,1846086362824884662,False,NaN,56 Ave & 92 St,classic,False,5962.04
4,103,True,40.760177,-73.984868,64f0f28c-bedc-42d5-b107-ecdd48fc30cd,64f0f28c-bedc-42d5-b107-ecdd48fc30cd,False,71,Broadway & W 48 St,classic,False,6708.04


### Writing a Pandas Dataframe in a MySQL Table

In [8]:
import sqlalchemy
from sqlalchemy import create_engine, text

conn_string = "mysql+pymysql://{user}:{password}@{host}/".format(
    host="db.ipeirotis.org", user="student", password="dwdstudent2015"
)

engine = create_engine(conn_string)

Once we have connected successfully, we need to create our database:

In [9]:
# Query to create a database
# In this example, we will try to create the (existing) database "public"
# But in general, we can give any name to the database
db_name = "public"
create_db_query = (
    f"CREATE DATABASE IF NOT EXISTS {db_name} DEFAULT CHARACTER SET 'utf8'"
)

# Create a database
with engine.connect() as con:
    con.execute(text(create_db_query))


# And lets switch to the database
with engine.connect() as con:
    con.execute(text(f"USE {db_name}"))

In [15]:
# To avoid conflicts between people writing in the same database, we add a random suffix in the tables
# We only create the variable once while running the notebook
import uuid

if "suffix" not in globals():
  suffix = str(uuid.uuid4())[:8]
print(suffix)

c4922960


### Create Table and Store Data in Database using the `to_sql` command

Then we create the table where we will store our data. Since we already have the data in a Pandas DataFrame, it is very easy to put the data in a database.

In [18]:
table_name = f"Stations_{suffix}"
# Create a table
# See http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html for the documentation

# This step is optional, but it is good practice to define explicitly the
# data types before storing things in a database. In many cases, this can be ommitted, though.

create_table_query = f"""CREATE TABLE IF NOT EXISTS {db_name}.{table_name}
                                (legacy_id int,
                                station_id varchar(100),
                                region_id int,
                                external_id varchar(50),
                                lat float,
                                lon float,
                                short_name varchar(10),
                                name varchar(250),
                                station_type varchar(10),
                                capacity int,
                                electric_bike_surcharge_waiver bool,
                                eightd_has_key_dispenser bool,
                                has_kiosk bool,
                                PRIMARY KEY(station_id)
                                )"""

with engine.connect() as con:
  con.execute(text(create_table_query))



In [19]:
with engine.connect() as conn:
  df.to_sql(
      name=table_name,
      schema=db_name,
      con=conn,
      if_exists="append",
      index=False
  )

AttributeError: ignored

In [20]:
# Once we have the data in the table, we also specify a primary key
# If we had FOREIGN KEYS we can add them in the same way
add_key_query = f"ALTER TABLE {table_name} ADD PRIMARY KEY(station_id)"
print(add_key_query)

with engine.connect() as con:
  con.execute(text(add_key_query))

ALTER TABLE Stations_c4922960 ADD PRIMARY KEY(station_id)


OperationalError: ignored

### Reading from a SQL Database in Python using the `read_sql` command in Pandas

We can similarly read from the database using Pandas

In [ ]:
query = text(f"SELECT * FROM {db_name}.{table_name}")
print(query)

In [ ]:
with engine.connect() as con:
  df2 = pd.read_sql(query, con=con)
df2.head(5)

### Export Data from Database to CSV or Excel

And remember that from Pandas it is also possible to export in other formats, such as Excel of CSV.

In [ ]:
# The necessary library to write in Excel
# !sudo pip3 install -U openpyxl

In [ ]:
df2.to_excel("citibike.xlsx")
df2.to_csv("citibike.csv")

### Cleanup

Finally, let's clean up and delete the table that we created

In [ ]:
drop_table_query = f"DROP TABLE IF EXISTS {db_name}.{table_name}"
print(drop_table_query)

with engine.connect() as con:
  con.execute(text(drop_table_query))



### Exercise

The `url_status = 'https://gbfs.citibikenyc.com/gbfs/en/station_status.json'` URL contains the status of the stations. Write code that reads the results from that API call, and then stores the data in a separate table. Add a "foreign key" constraint from the Status table to the Stations table that we created above.